In [1]:
import sys, os
sys.path.append("../../")
import pandas as pd
import category_encoders as ce
from tempfile import TemporaryDirectory
import uuid
import logging

import reco_utils.recommender.lightgbm.lightgbm_utils as lgb_utils
import reco_utils.dataset.criteo as criteo

import azureml.core

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
import azureml.dataprep as dprep

# Check core SDK version number - based on build number of preview/master.
print("SDK version:", azureml.core.VERSION)

username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user').split("@")[0]
print("Your username is {0}".format(username))

SDK version: 1.0.18
Your username is nikhilj

In [2]:
SIZE = "sample"

nume_cols = ["I" + str(i) for i in range(1, 14)]
cate_cols = ["C" + str(i) for i in range(1, 27)]
label_col = "Label"

header = [label_col] + nume_cols + cate_cols
with TemporaryDirectory() as tmp:
    all_data = criteo.load_pandas_df(size=SIZE, local_cache_path=tmp, header=header)

0.00B [00:00, ?B/s] 0%| | 0.00/8.79M [00:00<?, ?B/s] 0%| | 16.4k/8.79M [00:00<01:14, 118kB/s] 1%| | 49.2k/8.79M [00:00<01:02, 139kB/s] 1%| | 106k/8.79M [00:00<00:49, 174kB/s]  3%|▎ | 221k/8.79M [00:00<00:37, 228kB/s] 5%|▌ | 442k/8.79M [00:00<00:27, 306kB/s] 10%|█ | 901k/8.79M [00:01<00:18, 421kB/s] 18%|█▊ | 1.56M/8.79M [00:01<00:12, 579kB/s] 28%|██▊ | 2.48M/8.79M [00:01<00:07, 797kB/s] 38%|███▊ | 3.34M/8.79M [00:01<00:05, 1.08MB/s] 48%|████▊ | 4.21M/8.79M [00:01<00:03, 1.43MB/s] 58%|█████▊ | 5.05M/8.79M [00:01<00:02, 1.86MB/s] 67%|██████▋ | 5.86M/8.79M [00:01<00:01, 2.33MB/s] 76%|███████▌ | 6.66M/8.79M [00:02<00:00, 2.84MB/s] 88%|████████▊ | 7.70M/8.79M [00:02<00:00, 3.48MB/s] 97%|█████████▋| 8.51M/8.79M [00:02<00:00, 3.95MB/s]8.79MB [00:02, 3.69MB/s]

In [3]:
all_data.head()

Out[ 5 ]: 
 Label I1 I2 I3 ... C23 C24 C25 C26
0 0 1.00 1 5.00 ... 3a171ecb c5c50484 e8b83407 9727dd16
1 0 2.00 0 44.00 ... 3a171ecb 43f13e8b e8b83407 731c3655
2 0 2.00 0 1.00 ... 3a171ecb 3b183c5c NaN NaN
3 0 nan 893 nan ... 3a171ecb 9117a34a NaN NaN
4 0 3.00 -1 nan ... 32c7478e b34f3128 NaN NaN

[5 rows x 40 columns]

In [4]:
# split data to 3 sets    
length = len(all_data)
train_data = all_data.loc[:0.8*length-1]
valid_data = all_data.loc[0.8*length:0.9*length-1]
test_data = all_data.loc[0.9*length:]

In [5]:
ord_encoder = ce.ordinal.OrdinalEncoder(cols=cate_cols)

def encode_csv(df, encoder, label_col, typ='fit'):
    if typ == 'fit':
        df = encoder.fit_transform(df)
    else:
        df = encoder.transform(df)
    y = df[label_col].values
    del df[label_col]
    return df, y

train_x, train_y = encode_csv(train_data, ord_encoder, label_col)
valid_x, valid_y = encode_csv(valid_data, ord_encoder, label_col, 'transform')
test_x, test_y = encode_csv(test_data, ord_encoder, label_col, 'transform')

print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))

Train Data Shape: X: (80000, 39); Y: (80000,).
Valid Data Shape: X: (10000, 39); Y: (10000,).
Test Data Shape: X: (10000, 39); Y: (10000,).

In [6]:
subscription_id = "your_subscription_id" #you should be owner or contributor
resource_group = "your_rg" #you should be owner or contributor
workspace_name = "automl-aml-ws" #your workspace name
workspace_region = "westus2" #your region

In [7]:
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

Deploying StorageAccount with name nikhiljastoragejaconxoa.
Deploying ContainerRegistry with name nikhiljaacroorbxmro.
Deployed ContainerRegistry with name nikhiljaacroorbxmro.
Deploying AppInsights with name nikhiljainsightsxtpjewom.
Deployed AppInsights with name nikhiljainsightsxtpjewom.
Deploying KeyVault with name nikhiljakeyvaultqpzgqmmc.
Deployed KeyVault with name nikhiljakeyvaultqpzgqmmc.
Deployed StorageAccount with name nikhiljastoragejaconxoa.
Deploying Workspace with name nikhilj-aml-ws.
Deployed Workspace with name nikhilj-aml-ws.
 Out[ 16 ]: 
{'identityPrincipalId': '116ed18d-1033-43bf-9537-f14cab4bf153',
 'creationTime': '2019-03-22T23:19:17.3297990+00:00',
 'description': '',
 'friendlyName': 'nikhilj-aml-ws',
 'keyVault': '/subscriptions/e91107f8-bd8c-4517-8c8b-14879a7dbec4/resourcegroups/nikhilj/providers/microsoft.keyvault/vaults/nikhiljakeyvaultqpzgqmmc',
 'workspaceid': '7ebdf7b2-80a7-48cc-8150-c0509730255a',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'id': '/subscriptions/e91107f8-bd8c-4517-8c8b-14879a7dbec4/resourceGroups/nikhilj/providers/Microsoft.MachineLearningServices/workspaces/nikhilj-aml-ws',
 'applicationInsights': '/subscriptions/e91107f8-bd8c-4517-8c8b-14879a7dbec4/resourcegroups/nikhilj/providers/microsoft.insights/components/nikhiljainsightsxtpjewom',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/e91107f8-bd8c-4517-8c8b-14879a7dbec4/resourcegroups/nikhilj/providers/microsoft.storage/storageaccounts/nikhiljastoragejaconxoa',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'containerRegistry': '/subscriptions/e91107f8-bd8c-4517-8c8b-14879a7dbec4/resourcegroups/nikhilj/providers/microsoft.containerregistry/registries/nikhiljaacroorbxmro',
 'name': 'nikhilj-aml-ws',
 'location': 'westus2'}

In [8]:
X_dflow = dprep.read_pandas_dataframe(train_x, temp_folder='/dbfs/tmp'+str(uuid.uuid4()))
y_dflow = dprep.read_pandas_dataframe(pd.DataFrame(train_y, columns=["Label"]), temp_folder='/dbfs/tmp'+str(uuid.uuid4()))

In [9]:
y_dflow.get_profile()

Out[ 30 ]: 
ColumnProfile:
 column_name: Label
 type: FieldType.INTEGER

 min: 0.0
 max: 1.0
 count: 80000.0
 missing_count: 0.0
 not_missing_count: 80000.0
 percent_missing: 0.0
 error_count: 0.0
 empty_count: 0.0


 Quantiles:
 0.1%: 0.0
 1%: 0.0
 5%: 0.0
 25%: 0.0
 50%: 0.0
 75%: 0.0
 95%: 1.0
 99%: 1.0
 99.9%: 1.0

 mean: 0.2244750000000003
 std: 0.41723872120990974
 variance: 0.17408815047688078
 skewness: 1.3206902520386534
 kurtosis: -0.2557804549570242

## Configure Automated ML

You can use these params.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**spark_context**|Spark Context object. for Databricks, use spark_context=sc|
|**max_concurrent_iterations**|Maximum number of iterations to execute in parallel. This should be <= number of worker nodes in your Azure Databricks cluster.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [11]:
# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-criteo'
project_folder = './sample_projects/automl-criteo'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

Out[ 34 ]: 
 
Experiment Name automl-criteo 
Location westus2 
Project Directory ./sample_projects/automl-criteo 
Resource Group nikhilj 
SDK version 1.0.18 
Subscription ID e91107f8-bd8c-4517-8c8b-14879a7dbec4
Workspace Name nikhilj-aml-ws

In [12]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors_regression.log',
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 60, #some runs may take 10+ mins hence limiting it for workshop
                             iterations = 30, #you may change this to a higher number and see what happens
                             #validation_size = 0.20, #for large datasets only and not needed for workshop
                             verbosity = logging.INFO,
                             max_concurrent_iterations = 2, #change it based on number of worker nodes
                             spark_context=sc, #databricks/spark related
                             n_cross_validations = 3, #(only needed for small datasets and if validation size is not set)
                             X = X_dflow,
                             y = y_dflow,
                             preprocess=True, #preprocess
                             path = project_folder)

In [13]:
local_run = experiment.submit(automl_config, show_output = False) # for higher runs please use show_output=False and use the below

In [14]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

Your experiment in Azure Portal: AutoML_31318b27-767c-4aa4-a97b-f9e851fe17de